In [39]:
import pandas as pd

In [233]:
csv_columns = ['PRIMARIO', 'FUERA DE SERVICIO', 'REPOSICIONN', 'TIPO DE EVENTO', 'ESTADO DEL CLIMA', 'OBSERVACION']

In [234]:
# leer archivos
df1 = pd.read_csv(r"../data/202001.csv", encoding="ISO-8859-1")

In [235]:
# Transformar datasets
df1n = df1.BARRIOS.str.rstrip(',').str.split(',').apply(pd.Series)    # break df.BARRIOS into columns
df1n.index = df1.set_index(csv_columns).index
df1 = df1n.stack().reset_index(csv_columns)    # stack and reset_index
df1.columns = csv_columns  + ['BARRIO']

In [291]:
df1['DURACION'] = df1[["FUERA DE SERVICIO", "REPOSICIONN"]].apply(lambda x: (pd.to_datetime(x[1], format="%d/%m/%Y %H:%M", errors='coerce') - pd.to_datetime(x[0], format="%d/%m/%Y %H:%M", errors='coerce')), axis=1)

In [259]:
# Normalizar BARRIO
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\r", " "))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.strip())
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Hiódromo.", "Hipódromo"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Madame Lych", "Madame Lynch"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Dr. Gaspar R. de Francia", "Dr. Gaspar Rodriguez de Francia"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. Francisco.S. López", "Mcal. Francisco. S. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mbocayty", "Mbocayaty"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cañada de Ybyray", "Cañada Ybyray"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazaret", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazarethh", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Carlos. A. López", "Carlos A. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.rstrip("."))



In [260]:
aux = list(df1['BARRIO'].unique())

In [292]:
df1

,PRIMARIO,FUERA DE SERVICIO,REPOSICIONN,TIPO DE EVENTO,ESTADO DEL CLIMA,OBSERVACION,BARRIO,DURACION
0,BCP2,6/01/2020 15:50,6/01/2020 17:49,Accidental,Adverso,Condiciones Climáticas Adversas.,Bella Vista,0 days 01:59:00
1,BCP2,6/01/2020 15:50,6/01/2020 17:49,Accidental,Adverso,Condiciones Climáticas Adversas.,Santo Domingo,0 days 01:59:00
2,BCP2,6/01/2020 15:50,6/01/2020 17:49,Accidental,Adverso,Condiciones Climáticas Adversas.,Manorá,0 days 01:59:00
0,BCP2,6/01/2020 21:45,6/01/2020 22:30,Accidental,Adverso,Ramas sobre Línea MT. Mena Porta y Pedro P.\rP...,Bella Vista,0 days 00:45:00
1,BCP2,6/01/2020 21:45,6/01/2020 22:30,Accidental,Adverso,Ramas sobre Línea MT. Mena Porta y Pedro P.\rP...,Santo Domingo,0 days 00:45:00
...,...,...,...,...,...,...,...,...
0,VAU8,10/01/2020 08:10,10/01/2020 09:00,Accidental,Adverso,Condiciones Climáticas Adversas.,Villa Aurelia,0 days 00:50:00
0,VAU8,13/01/2020 17:38,13/01/2020 18:11,Accidental,Adverso,Transformador averiado. Rivarola Matto y Lomas...,Villa Aurelia,0 days 00:33:00
0,VAU8,13/01/2020 22:52,13/01/2020 23:02,Accidental,Normal,"Abierto Interruptor, para efectuar cierre de P...",Villa Aurelia,0 days 00:10:00
0,VAU8,16/01/2020 17:35,16/01/2020 19:05,Accidental,Normal,Aislador roto y pájaro en contacto con Línea M...,Villa Aurelia,0 days 01:30:00


In [262]:
AUX = df1.groupby(['TIPO DE EVENTO', 'BARRIO'])['BARRIO'].count()

In [263]:
AUX[["Accidental"]].sort_values(ascending=False)

TIPO DE EVENTO  BARRIO        
Accidental      Mburucuyá         26
                Obrero            21
                Itay              20
                Madame Lynch      20
                Villa Aurelia     18
                                  ..
                San Cayetano       2
                La Encarnación     2
                Santa Rosa         2
                Las Mercedes       1
                Nazareth           1
Name: BARRIO, Length: 69, dtype: int64

In [264]:
AUX[["Programado"]].sort_values(ascending=False)

TIPO DE EVENTO  BARRIO                         
Programado      Mcal. Estigarribia                 4
                San Antonio                        3
                Recoleta                           3
                Los Laureles                       3
                Ita Pyta Punta                     3
                Dr. Gaspar Rodriguez de Francia    3
                Villa Aurelia                      2
                Tembetary                          2
                Itay                               2
                Hipódromo                          2
                San Pablo                          1
                San Jorge                          1
                Nazareth                           1
                La Encarnación                     1
Name: BARRIO, dtype: int64

In [293]:
aux3 = df1[['BARRIO', 'DURACION']]

In [294]:
aux4 = df1.sort_values(by=['DURACION'], ascending=False)

In [295]:
aux4['duracion seg'] = aux4['DURACION'].apply(lambda x: x.seconds)

In [297]:
aux4.sort_values(by=["DURACION"], ascending=False)

,PRIMARIO,FUERA DE SERVICIO,REPOSICIONN,TIPO DE EVENTO,ESTADO DEL CLIMA,OBSERVACION,BARRIO,DURACION,duracion seg
0,PBO20,29/01/2020 20:20,30/01/2020 17:18,Accidental,Adverso,Condiciones Climáticas Adversas.,Botánico,0 days 20:58:00,75480.0
2,REP5,30/01/2020 00:10,30/01/2020 12:55,Accidental,Adverso,Condiciones Climáticas Adversas.,Pinoza,0 days 12:45:00,45900.0
0,REP5,30/01/2020 00:10,30/01/2020 12:55,Accidental,Adverso,Condiciones Climáticas Adversas.,Tte. Silvio Petirosi,0 days 12:45:00,45900.0
1,REP5,30/01/2020 00:10,30/01/2020 12:55,Accidental,Adverso,Condiciones Climáticas Adversas.,San Vicente,0 days 12:45:00,45900.0
0,BPA10,29/01/2020 18:14,30/01/2020 05:44,Accidental,Adverso,Condiciones Climáticas Adversas.,Itay,0 days 11:30:00,41400.0
...,...,...,...,...,...,...,...,...,...
2,VAU11,29/01/2020 18:09,Recierre,Accidental,Adverso,Condiciones Climáticas Adversas.,Villa Aurelia,NaT,NaN
0,VAU3,6/01/2020 15:27,Recierre,Accidental,Adverso,Condiciones Climáticas Adversas.,Villa Aurelia,NaT,NaN
1,VAU3,6/01/2020 15:27,Recierre,Accidental,Adverso,Condiciones Climáticas Adversas.,San Pablo,NaT,NaN
0,VAU3,31/01/2020 16:51,Recierre,Accidental,Normal,No establecido.,Villa Aurelia,NaT,NaN


In [290]:
tdelta2 = pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M") - pd.to_datetime("6/01/2020 23:54", format="%d/%m/%Y %H:%M")
tdelta2

Timedelta('0 days 01:18:00')

In [288]:
pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M")

Timestamp('2020-01-07 01:12:00')